<a href="https://colab.research.google.com/github/sugarforever/pydantic-tutorials/blob/main/pydantic_get_started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Started with PyDantic AI

Ever used FastAPI, LangChain, or the OpenAI Python SDK? Then you've already used [Pydantic](https://pydantic.dev/) under the hood. Pydantic is Python's go-to library for data validation, used by thousands of packages to ensure data matches expected types and formats.
What makes Pydantic special is its use of standard Python type hints. Instead of learning a new syntax, you just write regular Python code with type annotations, and Pydantic handles the validation:

```python
from pydantic import BaseModel

class User(BaseModel):
    name: str
    age: int
    email: str

json_string = '{"name": "John Doe", "age": 30, "email": "john@example.com"}'
user = User.parse_raw(json_string)

```

[Pydantic AI](https://ai.pydantic.dev/) builds on this foundation to make building AI applications just as straightforward. Created by the Pydantic team, it provides a framework for working with large language models (LLMs) that feels natural to Python developers.

Key features of Pydantic AI:
- Works with major LLM providers (OpenAI, Anthropic, Gemini, Groq)
- Uses standard Python for control flow and composition
- Validates AI responses using Pydantic models
- Supports streaming responses with validation
- Includes built-in debugging and monitoring

In this tutorial, we'll explore how to use Pydantic AI to build reliable AI applications using familiar Python patterns. Let's get started!

## Installation

The only Python package you need for now is `pydantic_ai`.

In [1]:
!pip install pydantic_ai -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 2.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.37.0 which is incompatible.


## Get Colab Environment Ready

To make the demo application run, we will also need `nest-asyncio`.

Next step is to set up environmental variable `OPENAI_API_KEY` so that the Pydantic AI Agents can pick it up in using OpenAI models.

In [2]:
!pip install nest-asyncio -qU

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

## Pydantic AI Agents

Let's start looking into some cool examples of Pydantic AI agents.

### The Simplest One

Chat with OpenAI `gpt-4o` straight away.

In [5]:
from pydantic_ai import Agent

In [6]:
agent = Agent("openai:gpt-4o")
response = agent.run_sync("Hey, dude!")
print(response.data)

Hey there! How can I assist you today?


### Agent with Static Prompt

In [7]:
agent = Agent("openai:gpt-4o", system_prompt="You can only speak Chinese")
response = agent.run_sync("Hey, dude!")
print(response.data)

你好！有什么我可以帮助你的吗？


### Agent with Dynamic Prompt

In [9]:
from pydantic_ai import Agent, RunContext

In [10]:
dynamic_prompt_agent = Agent("openai:gpt-4o")

@dynamic_prompt_agent.system_prompt
def set_agent_name(ctx: RunContext[str]) -> str:
    return f"Your name is {ctx.deps}."

response = dynamic_prompt_agent.run_sync("Hey, dude! Who are you?", deps="Jarvis")
print(response.data)

Hey there! I'm Jarvis, your AI assistant. How can I help you today?


### Agent with Dependency Type

In [11]:
from dataclasses import dataclass

@dataclass
class Player:
    name: str
    goals: int


agent = Agent(
    'openai:gpt-4o',
    deps_type=Player,
    result_type=bool,
)

@agent.system_prompt
def add_player_name(ctx: RunContext[Player]) -> str:
    player_name = ctx.deps.name
    return f"The player's name is {player_name}."

@agent.system_prompt
def add_player_goals(ctx: RunContext[Player]) -> str:
    goals = ctx.deps.goals
    return f"The player's goals so far is {goals}."

response = agent.run_sync("Hey, dude! Does the player ever score a goal?", deps=Player(name="Messi", goals=2))
print(response.data)

response = agent.run_sync("Hey, dude! Does the player ever score a goal?", deps=Player(name="Ronaldo", goals=0))
print(response.data)

True
False


### Agent with Function Tools

Function tools provide a mechanism for models to retrieve extra information to help them generate a response.

Developers use decorators `@agent.tool_plain` or `@agent.tool` to define tools.

In [13]:
agent = Agent('openai:gpt-4o')

@agent.tool
def get_player_goals(ctx: RunContext[str], player_name: str) -> str:
    print(f"Getting the goals of player {player_name} so far")
    if player_name == 'Messi':
        return '2'
    elif player_name == 'Ronaldo':
        return '100'
    else:
        return '0'

response = agent.run_sync("Let me know if Ronaldo scored so far")
print(response.data)

Getting the goals of player Ronaldo so far
Ronaldo has scored 100 goals so far.


In [14]:
response.all_messages()

[UserPrompt(content='Let me know if Ronaldo scored so far', timestamp=datetime.datetime(2024, 12, 13, 21, 28, 29, 504173, tzinfo=datetime.timezone.utc), role='user'),
 ModelStructuredResponse(calls=[ToolCall(tool_name='get_player_goals', args=ArgsJson(args_json='{"player_name":"Ronaldo"}'), tool_id='call_31XYZmv6dW8mOsEGatfpP7Zi')], timestamp=datetime.datetime(2024, 12, 13, 21, 28, 29, tzinfo=datetime.timezone.utc), role='model-structured-response'),
 ToolReturn(tool_name='get_player_goals', content='100', tool_id='call_31XYZmv6dW8mOsEGatfpP7Zi', timestamp=datetime.datetime(2024, 12, 13, 21, 28, 30, 253595, tzinfo=datetime.timezone.utc), role='tool-return'),
 ModelTextResponse(content='Ronaldo has scored 100 goals so far.', timestamp=datetime.datetime(2024, 12, 13, 21, 28, 30, tzinfo=datetime.timezone.utc), role='model-text-response')]

In [15]:
response = agent.run_sync("Let me know if Saka scored so far")
print(response.data)

Getting the goals of player Saka so far
Saka has not scored any goals so far.


In [16]:
response.all_messages()

[UserPrompt(content='Let me know if Saka scored so far', timestamp=datetime.datetime(2024, 12, 13, 21, 30, 3, 734955, tzinfo=datetime.timezone.utc), role='user'),
 ModelStructuredResponse(calls=[ToolCall(tool_name='get_player_goals', args=ArgsJson(args_json='{"player_name":"Saka"}'), tool_id='call_gsWKZgcj67OIi2D7F1UfgMMx')], timestamp=datetime.datetime(2024, 12, 13, 21, 30, 3, tzinfo=datetime.timezone.utc), role='model-structured-response'),
 ToolReturn(tool_name='get_player_goals', content='0', tool_id='call_gsWKZgcj67OIi2D7F1UfgMMx', timestamp=datetime.datetime(2024, 12, 13, 21, 30, 4, 370336, tzinfo=datetime.timezone.utc), role='tool-return'),
 ModelTextResponse(content='Saka has not scored any goals so far.', timestamp=datetime.datetime(2024, 12, 13, 21, 30, 4, tzinfo=datetime.timezone.utc), role='model-text-response')]